<a href="https://colab.research.google.com/github/Elakiyasuresh/Elakiyasuresh/blob/main/infotect_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask-ngrok opencv-python-headless torch torchvision pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os

os.makedirs("model", exist_ok=True)

from google.colab import files
uploaded = files.upload()  # Upload your `unet_model.pth` here


Saving dataset.zip to dataset.zip


In [3]:
import shutil
shutil.move(list(uploaded.keys())[0], "model/unet_model.pth")


'model/unet_model.pth'

In [4]:
import os

os.makedirs("model.py", exist_ok=True)

In [5]:
# Save as model.py
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.down1 = DoubleConv(3, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(128, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.upconv2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.upconv1 = DoubleConv(128, 64)
        self.final = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        d1 = self.down1(x)
        p1 = self.pool1(d1)
        d2 = self.down2(p1)
        p2 = self.pool2(d2)
        bn = self.bottleneck(p2)
        u2 = self.up2(bn)
        uc2 = self.upconv2(torch.cat([u2, d2], dim=1))
        u1 = self.up1(uc2)
        uc1 = self.upconv1(torch.cat([u1, d1], dim=1))
        return torch.sigmoid(self.final(uc1))


In [6]:
import os

os.makedirs("app.py", exist_ok=True)

In [17]:
# Save as app.py
from flask import Flask, render_template, request
import os, cv2, torch
import numpy as np
from PIL import Image
#from model import UNet
from torchvision import transforms
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

UPLOAD_FOLDER = 'static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet().to(device)
#model.load_state_dict(torch.load("model/unet_model.pth", map_location=device))
model.eval()

transform = transforms.Compose([
    transforms.ToTensor(),
])

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        image = request.files['image']
        path = os.path.join(UPLOAD_FOLDER, image.filename)
        image.save(path)

        img = cv2.imread(path)
        img_resized = cv2.resize(img, (128, 128))
        tensor = transform(img_resized).unsqueeze(0).to(device)

        with torch.no_grad():
            pred = model(tensor)[0][0].cpu().numpy()

        mask = (pred > 0.5).astype(np.uint8) * 255
        mask_path = os.path.join(UPLOAD_FOLDER, "mask_" + image.filename)
        cv2.imwrite(mask_path, mask)

        return render_template("index.html", image=image.filename, mask="mask_" + image.filename)
    return render_template("index.html")

app.run()

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

RuntimeError: Expected hasRecord("version") to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [8]:
import os

# Remove the directory named model.py if it exists
if os.path.isdir("model.py"):
    os.rmdir("model.py")

# Save the UNet class as model.py
model_code = """
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.down1 = DoubleConv(3, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(128, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.upconv2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.upconv1 = DoubleConv(128, 64)
        self.final = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        d1 = self.down1(x)
        p1 = self.pool1(d1)
        d2 = self.down2(p1)
        p2 = self.pool2(d2)
        bn = self.bottleneck(p2)
        u2 = self.up2(bn)
        uc2 = self.upconv2(torch.cat([u2, d2], dim=1))
        u1 = self.up1(uc2)
        uc1 = self.upconv1(torch.cat([u1, d1], dim=1))
        return torch.sigmoid(self.final(uc1))
"""

with open("model.py", "w") as f:
    f.write(model_code)

Now I will create the HTML file for the Flask app.

In [9]:
# Create the templates directory
os.makedirs("templates", exist_ok=True)

# Save the HTML content as index.html
html_content = """
<!doctype html>
<html>
<head>
    <title>Image Segmentation</title>
</head>
<body>
    <h1>Upload an image for segmentation</h1>
    <form method="post" enctype="multipart/form-data">
        <input type="file" name="image" accept="image/*">
        <input type="submit" value="Upload">
    </form>

    {% if image %}
    <h2>Original Image</h2>
    <img src="{{ url_for('static', filename='uploads/' + image) }}" width="256">
    <h2>Segmentation Mask</h2>
    <img src="{{ url_for('static', filename='uploads/' + mask) }}" width="256">
    {% endif %}
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html_content)

In [11]:
# Save as templates/index.html
!mkdir -p templates

with open("templates/index.html", "w") as f:
    f.write('''
<!DOCTYPE html>
<html>
<head><title>Disaster Segmentation</title></head>
<body>
    <h2>Upload Disaster Image</h2>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="image" required>
        <input type="submit" value="Segment">
    </form>

    {% if image %}
        <h3>Original:</h3>
        <img src="/static/uploads/{{ image }}" width="300"><br>
        <h3>Mask:</h3>
        <img src="/static/uploads/{{ mask }}" width="300">
    {% endif %}
</body>
</html>
''')


In [12]:
os.makedirs("static/uploads", exist_ok=True)


In [13]:
!python3 app.py


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


/usr/bin/python3: can't find '__main__' module in '/content/app.py'
